In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = None     ## Columns
pd.options.display.max_rows = None        ## Rows
import numpy as np


import sklearn
from sklearn import model_selection # for splitting into train and test
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from sklearn import linear_model # for logistic model



from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

df = pd.read_csv('listings.csv', sep = ';')
df.loc[df.Listing_Type=='Good','Listing_Type']=1
df.loc[df.Listing_Type=='Bad','Listing_Type']=0
df= df[0:100000]
df.shape


text_col=['Name','Summary','Space','Description','Neighborhood_Overview','Notes','Transit','Interaction','House_Rules','Host_About','Amenities']
df= df.drop(columns=text_col)
df.shape



## DROP Host_Name , Experiences_Offered , Unnamed: 0 , ID ,
# drop due to missing value - Access , Host_Response_Time , Host_Response_Rate ,Host_Acceptance_Rate, Host_Neighbourhood , Neighbourhood ,Neighbourhood_Group_Cleansed,Square_Feet,
# Weekly_Price , Monthly_Price , Security_Deposit , Cleaning_Fee , Has_Availability, First_Review , Last_Review , Reviews_per_Month , 
# Rating 

miss_val = ['Experiences_Offered' , 'Access' , 'Host_Response_Time' , 'Host_Response_Rate' ,'Host_Acceptance_Rate', 'Host_Neighbourhood' , 'Neighbourhood' ,'Neighbourhood_Group_Cleansed','Square_Feet','Weekly_Price' , 'Monthly_Price' , 'Security_Deposit' , 'Cleaning_Fee' , 'Has_Availability', 'First_Review' , 'Last_Review' , 'Reviews_per_Month' , 'Rating' ]
df= df.drop(columns=miss_val)

id_col = ['Host_Name'  , 'Unnamed: 0' , 'ID']
df= df.drop(columns=id_col)

df.shape



dateList=['Host_Since','Calendar_last_Scraped']
df[dateList].head()

# converting above 3 columns to datetime (from object type)

for dateColumn in dateList:
    print(dateColumn)
    df[dateColumn] = pd.to_datetime(df[dateColumn],format='%Y-%m-%d',errors='coerce')
    

for dateColumn in dateList:
    print(dateColumn)
#     df[dateColumn+'_YEAR']=df[dateColumn].dt.year
#     df[dateColumn+'_MONTH']=df[dateColumn].dt.month
#     df[dateColumn+'_WEEK']=df[dateColumn].dt.week
#     df[dateColumn+'_DAY']=df[dateColumn].dt.day
#     df[dateColumn+'_DAYOFWEEK']=df[dateColumn].dt.dayofweek
    df[dateColumn+'_DAYSSINCETODAY']=(pd.datetime.now()-df['Host_Since']).dt.days



df= df.drop(columns=['Host_Since','Calendar_last_Scraped'])
# unwanted_dates = ['Host_Since','Host_Since_YEAR' , 'Host_Since_MONTH', 'Host_Since_WEEK' , 'Host_Since_DAY' , 'Host_Since_DAYOFWEEK'  ]
# df= df.drop(columns=unwanted_dates)
# unwanted_dates = ['Calendar_last_Scraped','Calendar_last_Scraped_YEAR' , 'Calendar_last_Scraped_YEAR', 'Calendar_last_Scraped_MONTH' , 'Calendar_last_Scraped_DAY' , 'Calendar_last_Scraped_DAYOFWEEK'  ]
# df= df.drop(columns=unwanted_dates)
df.shape



add_col = ['Host_Location','Street','Neighbourhood_Cleansed','City','State','Zipcode','Market','Smart_Location','Country_Code','Country']
df= df.drop(columns=add_col)

df= df.drop(columns='Geolocation')     ### REDUNTENT 
df= df.drop(columns='Features')    ### TO BE SEEN AFTER WARDS
df= df.drop(columns=['License','Jurisdiction_Names','Calendar_Updated'])
df.shape



# Imputation



numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numColumns=df.select_dtypes(include=numerics).columns
numColumns

df['Extra_People']=df['Extra_People'].fillna(0)

df[numColumns]=df[numColumns].fillna(df.mean())
#numColumns1 = ['Host_Listings_Count' , 'Property_Type' ,'Room_Type' ,'Bed_Type' , 'Guests_Included' ,'Maximum_Nights','Cancellation_Policy']
#df[numColumns1]=df[numColumns1].fillna(df.mean())

df.shape


df['Host_Listings_Count']=df['Host_Listings_Count'].fillna(0)

df['Property_Type'] = df['Property_Type'].fillna(df['Property_Type'].mode()[0])

df['Room_Type'] = df['Room_Type'].fillna(df['Room_Type'].mode()[0])

df['Accommodates'] = df['Accommodates'].fillna(df['Accommodates'].mode()[0])

df['Bed_Type'] = df['Bed_Type'].fillna(df['Bed_Type'].mode()[0])

df['Guests_Included'] = df['Guests_Included'].fillna(df['Guests_Included'].mode()[0])

df['Maximum_Nights'] = df['Maximum_Nights'].fillna(df['Maximum_Nights'].mode()[0])

df['Cancellation_Policy']=df['Cancellation_Policy'].str.extract(r'(flexible|moderate|strict|no)')
df['Cancellation_Policy'] = df['Cancellation_Policy'].fillna(df['Cancellation_Policy'].mode()[0])


df.shape

df.isnull().sum()











## ONe hot encode
dfnumhot=pd.get_dummies(df[['Property_Type','Room_Type','Bed_Type','Cancellation_Policy']],
                                 prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)

dropped_column=['Property_Type','Room_Type','Bed_Type','Cancellation_Policy']
df= df.drop(columns=dropped_column) 


## Concat
df = pd.concat([df, dfnumhot], axis=1)
df.shape



#df_hot.select_dtypes(include='object').columns

df.Host_Listings_Count.value_counts()
df.Host_Listings_Count[df.Host_Listings_Count == 'Private room'] = 1
df.Host_Listings_Count[df.Host_Listings_Count == 'Entire home/apt'] = 1

df.Accommodates[df.Accommodates == 'yesterday'] = 2
df.Accommodates[df.Accommodates == '7 weeks ago'] = 2
df.Accommodates[df.Accommodates == 'a week ago'] = 2

df.Guests_Included[df.Guests_Included == 'moderate'] = 1
df.Guests_Included[df.Guests_Included == 'strict'] = 1



df.Maximum_Nights[df.Maximum_Nights == '51.1975959128, 4.4295469314'] = 1
df.shape



df['Host_Listings_Count'] = pd.to_numeric(df['Host_Listings_Count'])
df['Accommodates'] = pd.to_numeric(df['Accommodates'])
df['Guests_Included'] = pd.to_numeric(df['Guests_Included'])

df['Maximum_Nights'] = pd.to_numeric(df['Maximum_Nights'])
df['Listing_Type'] = pd.to_numeric(df['Listing_Type'])

df.shape



df.dtypes[df.dtypes == "O"]





dataset = pd.read_csv('listings.csv', sep = ';' , usecols= ['Amenities','Features'])
dataset = dataset[0:100000]
print(df.shape)
print(dataset.shape)

dataset.head()

dataset['Amenities']=dataset['Amenities'].str.replace(' ','_')
dataset['Amenities']=dataset['Amenities'].str.replace('\"','')
dataset['Amenities']=dataset['Amenities'].str.replace(',',' ')
dataset['Amenities']=dataset['Amenities'].fillna('')
dataset['Amenities']=dataset['Amenities'].str.replace('-','_')


dataset['Features']=dataset['Features'].str.replace(' ','_')
dataset['Features']=dataset['Features'].str.replace('\"','')
dataset['Features']=dataset['Features'].str.replace(',',' ')
dataset['Features']=dataset['Features'].fillna('')
dataset['Features']=dataset['Features'].str.replace('-','_')
print(df.shape)
print(dataset.shape)

from sklearn.feature_extraction.text import CountVectorizer

vectFeatures = CountVectorizer()
vectFeatures.fit(dataset['Features'])

featuresSplit=vectFeatures.transform(dataset['Features'])
#featuresSplit.toarray()

vectAmenities = CountVectorizer()
vectAmenities.fit(dataset['Amenities'])

amenitiesSplit=vectAmenities.transform(dataset['Amenities'])
#amenitiesSplit.toarray()
print(df.shape)
print(dataset.shape)

featuresCSV=pd.DataFrame(featuresSplit.toarray(),columns=vectFeatures.get_feature_names())

amenitiesCSV=pd.DataFrame(amenitiesSplit.toarray(),columns=vectAmenities.get_feature_names())

csvDF=pd.concat([featuresCSV,amenitiesCSV],axis=1)
print(csvDF.shape)
print(df.shape)
print(dataset.shape)

df=pd.concat([df.reset_index(drop=True),csvDF],axis=1)
df.shape

df.shape





df_text_subset  = pd.read_csv('df_text_subset.csv')
df_text_subset = df_text_subset[0:100000]

df_text_subset.shape

df = pd.concat([df, df_text_subset], axis=1)
df.shape

#Listing_Type
df = df.dropna(axis=0, subset=['Listing_Type'])
df.shape

df.shape

Host_Since
Calendar_last_Scraped
Host_Since
Calendar_last_Scraped


C:\Users\Jatin\Anaconda3\lib\site-packages\ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Jatin\Anaconda3\lib\site-packages\ipykernel_launcher.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Jatin\Anaconda3\lib\site-packages\ipykernel_launcher.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Jatin\Anaconda3\lib\site-packages\ipykernel_launcher.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

(100000, 81)
(100000, 2)
(100000, 81)
(100000, 2)
(100000, 81)
(100000, 2)
(100000, 131)
(100000, 81)
(100000, 2)


(99992, 223)

In [ ]:
#######################################################################################################################
#########################################################################################################################

# MOdel

In [4]:
# Split-out validation dataset
X = df.select_dtypes(include=numerics).drop(columns=['Listing_Type']).values
Y = df['Listing_Type'].values
scaler = MinMaxScaler()
X=scaler.fit_transform(X)
X_Columns=df.select_dtypes(include=numerics).drop(columns=['Listing_Type']).columns
validation_size = 0.30
seed = 100
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [35]:
print('X_train\t',len(X_train))
print('X_test\t',len(X_test))
print('Y_train\t',len(Y_train))
print('Y_test\t',len(Y_test))


X_train	 69994
X_test	 29998
Y_train	 69994
Y_test	 29998



## Logistic

In [42]:
## BASE LINE
# Algorithms
model_LR=sklearn.linear_model.LogisticRegression(class_weight='balanced', penalty='l1')


# fitting model
model_LR.fit(X_train,Y_train)

#### Making Prediction ####

trainResult_LR=model_LR.predict(X_train)
testResult_LR=model_LR.predict(X_test)

# Evaluation

### Accuracy score
### Confusion Matrix
### Classification Report


# Train

print(sklearn.metrics.accuracy_score(Y_train, trainResult_LR))
print(sklearn.metrics.confusion_matrix(Y_train, trainResult_LR))
print(sklearn.metrics.classification_report(Y_train, trainResult_LR))
print('--------------------------------------------------------------')

# Test
print(sklearn.metrics.accuracy_score(Y_test, testResult_LR))
print(sklearn.metrics.confusion_matrix(Y_test, testResult_LR))
print(sklearn.metrics.classification_report(Y_test, testResult_LR))
print('____________________________________________________________')
if sklearn.metrics.accuracy_score(Y_train, trainResult_LR) > sklearn.metrics.accuracy_score(Y_test, testResult_LR):

    print('MODEL IS NOT OVERFITTING')
else:
    print('MODEL IS  OVERFITTING')


C:\Users\Jatin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7669943137983256
[[ 5969   453]
 [15856 47716]]
              precision    recall  f1-score   support

         0.0       0.27      0.93      0.42      6422
         1.0       0.99      0.75      0.85     63572

   micro avg       0.77      0.77      0.77     69994
   macro avg       0.63      0.84      0.64     69994
weighted avg       0.92      0.77      0.81     69994

--------------------------------------------------------------
0.7601173411560771
[[ 2461   214]
 [ 6982 20341]]
              precision    recall  f1-score   support

         0.0       0.26      0.92      0.41      2675
         1.0       0.99      0.74      0.85     27323

   micro avg       0.76      0.76      0.76     29998
   macro avg       0.63      0.83      0.63     29998
weighted avg       0.92      0.76      0.81     29998

____________________________________________________________
MODEL IS OVERFITTING


In [ ]:
# dataset.loc[dataset.Listing_Type=='Good','Listing_Type']=1

# dataset.loc[dataset.Listing_Type=='Bad','Listing_Type']=0